# 🔁 State-Based LSTM Autoencoder Training

This notebook splits sensor data by machine state (`run`, `idle`, `off`), trains an LSTM Autoencoder on each subset, and saves the models separately.

## 📦 Step 1: Import Required Libraries

In [2]:

import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, TimeDistributed, Dense, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping


## 📥 Step 2: Load and Preview Data

In [3]:

df = pd.read_csv("../test_csv/motor_monitor_0_reshaped.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()


,timestamp,power,rpm,temperature,vibration,is_anomaly,state,label
0,2025-04-07 00:00:00,5.022329,2.653643,22.0,7.523581,False,off,Normal
1,2025-04-07 00:01:00,5.077981,2.226890,22.0,7.324660,False,off,Normal
2,2025-04-07 00:02:00,4.833379,2.293508,22.0,7.432946,False,off,Power_outage
3,2025-04-07 00:03:00,6.071584,2.802438,22.0,7.827574,False,off,Normal
4,2025-04-07 00:04:00,4.644964,2.028206,22.0,7.241896,False,off,Power_outage


## 🔀 Step 3: Split Data by Machine State

In [4]:

df_run = df[df['state'] == 'run'].reset_index(drop=True)
df_idle = df[df['state'] == 'idle'].reset_index(drop=True)
df_off = df[df['state'] == 'off'].reset_index(drop=True)

print(f"Run state samples: {len(df_run)}")
print(f"Idle state samples: {len(df_idle)}")
print(f"Off state samples: {len(df_off)}")


Run state samples: 1184
Idle state samples: 105
Off state samples: 150


## 🧱 Step 4: Define Utility Functions for Training

In [5]:

def create_sequences(data, seq_length=30):
    return np.array([data[i:i+seq_length] for i in range(len(data)-seq_length)])

def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='relu', return_sequences=True),
        Dropout(0.2),
        TimeDistributed(Dense(input_shape[1]))
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


## 🧠 Step 5: Train LSTM per State

In [6]:

def train_state_model(df_state, state_name):
    scaler = MinMaxScaler()
    features = df_state[['power', 'rpm', 'temperature', 'vibration']]
    scaled = scaler.fit_transform(features)
    
    sequences = create_sequences(pd.DataFrame(scaled))
    model = build_lstm_model((sequences.shape[1], sequences.shape[2]))
    
    print(f"Training model for state: {state_name}")
    model.fit(sequences, sequences, epochs=20, batch_size=32,
              validation_split=0.1, callbacks=[EarlyStopping(patience=3)])
    
    model.save(f"../test_cases/saved_models/lstm_{state_name}.h5")
    print(f"Model saved as lstm_{state_name}.h5")

train_state_model(df_run, 'run')
train_state_model(df_idle, 'idle')
train_state_model(df_off, 'off')


Training model for state: run
Epoch 1/20


c:\Users\Thomas\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.3016 - val_loss: 0.1060
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0872 - val_loss: 0.0579
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0586 - val_loss: 0.0398
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0454 - val_loss: 0.0291
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0364 - val_loss: 0.0214
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0311 - val_loss: 0.0184
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0266 - val_loss: 0.0141
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0237 - val_loss: 0.0118
Epoch 9/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0214 - val_loss: 0.0102
Epoch 10/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0197 - val_loss: 0.0086
Epoch 11/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0186 - val_loss: 0.0076
Epoch 12/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0172 - val_l

Model saved as lstm_run.h5
Training model for state: idle
Epoch 1/20


c:\Users\Thomas\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - loss: 0.2336 - val_loss: 0.2126
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2086 - val_loss: 0.1887
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1861 - val_loss: 0.1657
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1618 - val_loss: 0.1376
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1340 - val_loss: 0.1059
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1112 - val_loss: 0.0860
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1017 - val_loss: 0.0714
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0858 - val_loss: 0.0721
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0856 - val_loss: 0.0696
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0812 - val_loss: 0.0641
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0775 - val_loss: 0.0595
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0717 - val_loss: 0.0564
Epoch 13/20

Model saved as lstm_idle.h5
Training model for state: off
Epoch 1/20


c:\Users\Thomas\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - loss: 0.2033 - val_loss: 0.1646
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1801 - val_loss: 0.1406
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1551 - val_loss: 0.1094
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1220 - val_loss: 0.0759
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0995 - val_loss: 0.0620
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0895 - val_loss: 0.0577
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0797 - val_loss: 0.0537
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0723 - val_loss: 0.0457
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0693 - val_loss: 0.0403
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0643 - val_loss: 0.0391
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0613 - val_loss: 0.0390
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0593 - val_loss: 0.0377
Epoch 13/20

Model saved as lstm_off.h5
